
<a href="https://colab.research.google.com/github/is-leeroy-jenkins/Halo-Kitty-Adventures/blob/main/sql/notebooks/access.ipynb" target="_parent">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🗂️ SQL: MS Access



*A Deep Dive into Jet/ACE SQL and VBA Integration*

## 🧭 Introduction

- Microsoft Access is not just a spreadsheet replacement — it’s a **relational database system** that uses a version of SQL known as **Jet/ACE SQL**.

- While the SQL syntax in Access looks similar to SQL Server or MySQL, it has unique behavior, functions, and data-type handling rules because it’s interpreted by the **Microsoft Access Database Engine (ACE)**.

#### This guide will teach you:

* How to write and understand SQL queries within Access

* How to execute queries using **VBA**

* How to combine Access forms, reports, and macros with SQL for automation and reporting

We’ll move gradually from basic query construction to advanced topics like parameter queries, joins, subqueries, and crosstab reports — all written in clean Access SQL.


## ⚙️ SQL Environment

### Where SQL Lives in Access

- [Download/Install](https://www.microsoft.com/en-us/download/details.aspx?id=54920)

Every Access database (`.accdb` or `.mdb`) has an underlying **database engine** (Jet for older versions, ACE for newer).
When you create a query in the **Query Design View**, Access actually builds an SQL statement behind the scenes.

You can view or edit that statement directly by switching to **SQL View**:

* Open the Query Designer.

* Select **View → SQL View** from the toolbar.

The **SQL View** window is where Access interprets and stores SQL commands.

## SQL Queries


| Query Type               | Purpose                                 | Returns Results? |
| ------------------------ | --------------------------------------- | ---------------- |
| **SELECT**               | Retrieves data.                         | ✅ Yes            |
| **INSERT INTO**          | Adds new records.                       | ❌ No             |
| **UPDATE**               | Modifies existing records.              | ❌ No             |
| **DELETE**               | Removes records.                        | ❌ No             |
| **MAKE-TABLE**           | Creates a new table from query results. | ❌ No             |
| **APPEND**               | Adds data to an existing table.         | ❌ No             |
| **CROSSTAB (TRANSFORM)** | Summarizes data in pivot-table format.  | ✅ Yes            |
| **UNION**                | Combines multiple datasets.             | ✅ Yes            |



## SQL in VBA

- Access’s **VBA environment** (Visual Basic for Applications) gives you full control over executing SQL.

- Two main approaches exist:

1. **DAO (Data Access Objects)** – the most direct interface to Access tables and queries.

2. **DoCmd methods** – used for running saved queries or executing SQL strings directly.

> This dual environment — SQL inside Access and SQL inside VBA — is what makes Access both beginner-friendly and powerful for automation.

Example:


In [ ]:

' Run an action query (no results returned)
CurrentDb.Execute "UPDATE Employees SET Salary = Salary * 1.05;", dbFailOnError


In [ ]:

' Open a recordset and read data
Dim rs As DAO.Recordset
Set rs = CurrentDb.OpenRecordset("SELECT FirstName, LastName FROM Employees;")
Do While Not rs.EOF
    Debug.Print rs!FirstName, rs!LastName
    rs.MoveNext
Loop
rs.Close


## SELECT STATEMENT

- SELECT statements do not change data in the database.

- SELECT is usually the first word in an SQL statement. Most SQL statements are either SELECT or SELECT…INTO statements.

- The minimum syntax for a SELECT statement is:

    ```     SELECT fields FROM table```

- You can use an asterisk (*) to select all fields in a table. The following example selects all of the fields in the Employees table.

In [ ]:
SELECT [predicate] { * | table.* | [table.]field1 [AS alias1] [, [table.]field2 [AS alias2] [, …]]} 
FROM tableexpression [, …] [IN externaldatabase] 
[WHERE… ] 
[GROUP BY… ] 
[HAVING… ] 
[ORDER BY… ] 
[WITH OWNERACCESS OPTION]

- If a field name is included in more than one table in the FROM clause, precede it with the table name and the . (dot) operator.

In [ ]:
SELECT Employees.Department, Employees.SupvName 
FROM Employees 
WHERE Employees.Department = "X";

## SELECT INTO Statement

In [ ]:
SELECT field1[, field2[, …]] 
INTO newtable [IN externaldatabase] 
FROM source

## Sub Queries

- You can use a subquery instead of an expression in the field list of a `SELECT `statement or in a `WHERE` or `HAVING` clause. 

- In a subquery, you use a `SELECT` statement to provide a set of one or more specific values to evaluate in the `WHERE` or `HAVING` clause expression.

- Use the `ANY` or `SOME` predicate, which are synonymous, to retrieve records in the main query that satisfy the comparison with any records retrieved in the subquery. 

In [ ]:
SELECT * FROM Products 
WHERE UnitPrice > ANY 
(SELECT UnitPrice FROM OrderDetails 
WHERE Discount >= .25);

- Use the `ALL` predicate to retrieve only those records in the main query that satisfy the comparison with all records retrieved in the subquery. 

- Use the `IN` predicate to retrieve only those records in the main query for which some record in the subquery contains an equal value. 

In [ ]:
SELECT * FROM Products 
WHERE ProductID IN 
(SELECT ProductID FROM OrderDetails 
WHERE Discount >= .25);

- Conversely, you can use NOT IN to retrieve only those records in the main query for which no record in the subquery contains an equal value.

- Use the `EXISTS` predicate (with the optional NOT reserved word) in true/false comparisons to determine whether the subquery returns any records.

- You can also use table name aliases in a subquery to refer to tables listed in a `FROM` clause outside the subquery.

In [ ]:
SELECT LastName,
FirstName, Title, Salary 
FROM Employees AS T1 
WHERE Salary >= (SELECT Avg(Salary) 
FROM Employees 
WHERE T1.Title = Employees.Title) Order by Title;

## UPDATE Statement

In [ ]:
UPDATE Orders 
SET OrderAmount = OrderAmount * 1.1, 
Freight = Freight * 1.03 
WHERE ShipCountry = 'UK';

## INSERT Statement

- You can use the `INSERT INTO` statement to add a single record to a table using the single-record append query syntax as shown above. In this case, your code specifies the name and value for each field of the record. 

- You must specify each of the fields of the record that a value is to be assigned to and a value for that field. 

- When you do not specify each field, the default value or Null is inserted for missing columns. 

- Records are added to the end of the table.

- You can also use `INSERT INTO` to append a set of records from another table or query by using the `SELECT` … `FROM` clause as shown above in the multiple-record append query syntax. In this case, the `SELECT` clause specifies the fields to append to the specified target table.

- The source or target table may specify a table or a query. If a query is specified, the Microsoft Access database engine appends records to any and all tables specified by the query.

- `INSERT INTO` is optional but when included, precedes the `SELECT` statement.

- If your destination table contains a primary key, make sure you append unique, non-Null values to the primary key field or fields; if you do not, the Microsoft Access database engine will not append the records.

- If you append records to a table with an `AutoNumber` field and you want to renumber the appended records, do not include the `AutoNumber` field in your query. Do include the `AutoNumber` field in the query if you want to retain the original values from the field.

- Use the `IN` clause to append records to a table in another database.

- To create a new table, use the `SELECT...INTO` statement instead to create a make-table query.

- To find out which records will be appended before you run the append query, first execute and view the results of a select query that uses the same selection criteria.

- An append query copies records from one or more tables to another. The tables that contain the records you append are not affected by the append query.

- Instead of appending existing records from another table, you can specify the value for each field in a single new record using the `VALUES` clause. If you omit the field list, the `VALUES` clause must include a value for every field in the table; otherwise, the` INSERT` operation will fail. Use an additional `INSERT INTO` statement with a `VALUES` clause for each additional record you want to create.

#### Single-record

In [ ]:
INSERT INTO target [(field1[, field2[, …]])] 
VALUES (value1[, value2[, …])

#### Multiple-record

In [ ]:
INSERT INTO target [(field1[, field2[, …]])] [IN externaldatabase] 
SELECT [source.]field1[, field2[, …] 
FROM tableexpression

## 🧱 SQL Basics

- The foundation of every SQL statement in Access is:

In [ ]:
SELECT field_list
FROM table_name
WHERE criteria
ORDER BY sort_order;

### Example

In [ ]:
SELECT FirstName, LastName, Department
FROM Employees
WHERE Department = "Finance"
ORDER BY LastName;



- This retrieves all Finance employees and sorts them alphabetically by last name.



## 🧠 Execution Order


- The **Jet/ACE engine** processes statements in a specific **logical order** that determines how results are built.

- Understanding this sequence explains many Access “mysteries,” such as why aliases aren’t recognized in the `WHERE` clause or why totals queries require the `HAVING` clause.

#### Logical Order of Execution

| Step  | Clause           | Description                                   |
| ----- | ---------------- | --------------------------------------------- |
| **1** | `FROM`           | Load tables and perform joins or subqueries.  |
| **2** | `WHERE`          | Filter individual rows (row-level filtering). |
| **3** | `GROUP BY`       | Group the remaining rows into categories.     |
| **4** | `HAVING`         | Filter groups based on aggregate results.     |
| **5** | `SELECT`         | Return specific columns or expressions.       |
| **6** | `ORDER BY`       | Sort the final result set.                    |
| **7** | `TOP / DISTINCT` | Apply record limits or remove duplicates.     |

### Example: Department Salary Analysis

In [ ]:
SELECT Department, AVG(Salary) AS AvgSalary
FROM Employees
WHERE HireDate >= #1/1/2020#
GROUP BY Department
HAVING AVG(Salary) > 85000
ORDER BY AvgSalary DESC;

**Execution flow:**

1. **FROM** — Access retrieves all records from `Employees`.

2. **WHERE** — Filters employees hired after January 1, 2020.

3. **GROUP BY** — Groups remaining employees by department.

4. **HAVING** — Keeps only groups with an average salary above $85,000.

5. **SELECT** — Produces two columns: `Department` and the calculated `AvgSalary`.

6. **ORDER BY** — Sorts results from highest to lowest average salary.

7. **TOP** (if present) — Would then limit the number of rows returned.


## Access Nuances

- **Access executes JOINs first**, even before evaluating `WHERE` filters.

- This means row combinations are formed before filtering — an important distinction when working with outer joins.

- **Aliases defined in `SELECT` cannot be used in `WHERE`** because the `WHERE` clause executes first.

-  You can use aliases in `ORDER BY` since it executes last.

- **`HAVING` is the only clause** that can reference aggregate functions such as `SUM()` or `AVG()`.

- **`DISTINCT` and `TOP`** are applied *after* ordering — which is why applying `TOP 10` to an unordered query can yield inconsistent results.

- **Totals Queries in Design View** correspond exactly to the `GROUP BY` → `HAVING` stages.

#### Why It Matters

| Common Confusion                                      | Explanation                                                             |
| ----------------------------------------------------- | ----------------------------------------------------------------------- |
| “Why does Access say my alias doesn’t exist?”         | Because the alias is created in `SELECT`, which runs after `WHERE`.     |
| “Why can’t I filter averages in WHERE?”               | Aggregates don’t exist yet; you must use `HAVING`.                      |
| “Why does changing JOIN type change my record count?” | Access executes joins before filtering, affecting which rows qualify.   |
| “Why does TOP 10 behave differently each run?”        | Without `ORDER BY`, Access picks arbitrary rows — add explicit sorting. |

## Logical vs. Physical Processing

- This order represents the **logical** flow of SQL — the conceptual sequence the Jet/ACE engine uses.

- Internally, Access may reorder or optimize steps for performance (e.g., pushing filters earlier, using indexes, or caching joined tables).

- Understanding the logical sequence is crucial for writing queries that behave predictably.

### Quick Reference Diagram

```
┌────────────────────────────────────┐
│ FROM → WHERE → GROUP BY → HAVING   │
│ → SELECT → ORDER BY → TOP/DISTINCT │
└────────────────────────────────────┘
```


## Understanding Each Clause






| Clause     | Purpose                                                 | Notes                                        |
| ---------- | ------------------------------------------------------- | -------------------------------------------- |
| `SELECT`   | Specifies which columns (fields) to return.             | You can also include calculated expressions. |
| `FROM`     | Indicates which table(s) to read from.                  | Supports joins and subqueries.               |
| `WHERE`    | Filters rows based on a condition.                      | Optional; works before grouping.             |
| `ORDER BY` | Sorts results ascending (`ASC`) or descending (`DESC`). | Access defaults to ascending.                |

- If you omit the `WHERE` clause, Access returns all records in the table — similar to “Select All”.



## 📅 Data Types and Literals in Access SQL



- Access SQL uses a simple but strict system for data representation.

| Data Type     | Example          | Notes                                       |
| ------------- | ---------------- | ------------------------------------------- |
| **Text**      | `"Smith"`        | Strings use double quotes or single quotes. |
| **Number**    | `42`, `3.14`     | No quotes needed.                           |
| **Date/Time** | `#1/1/2025#`     | Date literals **must** be enclosed in `#`.  |
| **Boolean**   | `True` / `False` | Stored internally as -1 and 0.              |



- Access always interprets dates in **U.S. format (MM/DD/YYYY)**, regardless of regional settings.

- If your system uses a different locale, still write `#12/31/2025#` (not `#31/12/2025#`).

#### Example:


In [ ]:
SELECT * FROM Orders
WHERE OrderDate >= #1/1/2025# AND Shipped = True;


## 🔍 Filtering with WHERE


- The `WHERE` clause refines which records appear in your results.

#### Comparison Operators

| Operator             | Description | Example                    |
| -------------------- | ----------- | -------------------------- |
| `=`                  | Equal to    | `WHERE City = "Boston"`    |
| `<>`                 | Not equal   | `WHERE Department <> "IT"` |
| `<`, `>`, `<=`, `>=` | Comparison  | `WHERE Salary >= 60000`    |

#### Combining Conditions


> Logical operators `AND`, `OR`, and `NOT` combine multiple conditions.



In [ ]:
SELECT * FROM Employees
WHERE Department = "Finance"
  AND Salary > 80000;


## Pattern Matching with LIKE


- Unlike most SQL dialects, Access uses `*` and `?` as wildcards (not `%` and `_`).

> Returns all cities beginning with “New” (e.g., *New York*, *Newark*).



In [ ]:

SELECT * FROM Customers
WHERE City LIKE "New*";



## Null Checks


- Because `NULL` represents “no value,” comparisons like `= NULL` will fail.
- Use `IS NULL` or `IS NOT NULL`:



In [ ]:

SELECT * FROM Orders
WHERE ShippedDate IS NULL;

## 🪶 Sorting and Aliases

Sorting results makes data easier to analyze or present in reports.


In [ ]:

SELECT LastName AS EmployeeLast, FirstName AS EmployeeFirst
FROM Employees
ORDER BY EmployeeLast ASC;



* `AS` assigns a friendly alias to a column name.
* By default, `ORDER BY` sorts ascending; append `DESC` for descending order.

## Table Aliases

Table aliases shorten long table names, especially in joins:

In [ ]:


SELECT e.FirstName, e.LastName, d.DepartmentName
FROM Employees AS e
INNER JOIN Departments AS d
ON e.DepartmentID = d.DepartmentID;



## 🧮 Calculated Fields and Built-In Functions

Access lets you compute values directly in queries using expressions and built-in functions.

### Example: Calculated Field

In [ ]:
SELECT FirstName, LastName, Salary, Salary * 1.05 AS NewSalary
FROM Employees;

Creates a new calculated column named **NewSalary**.

### Common Built-In Functions

| Category        | Function                              | Example                              | Description                      |
| --------------- | ------------------------------------- | ------------------------------------ | -------------------------------- |
| **String**      | `LEFT(text, n)`                       | `LEFT(LastName, 3)`                  | Returns leftmost `n` characters. |
|                 | `LEN(text)`                           | `LEN(LastName)`                      | Counts string length.            |
| **Date/Time**   | `DateAdd(interval, n, date)`          | `DateAdd("m", 3, OrderDate)`         | Adds months, days, or years.     |
|                 | `Now()`                               | –                                    | Current date and time.           |
| **Math**        | `Round(x, n)`                         | `Round(Salary, 0)`                   | Rounds numbers.                  |
| **Conditional** | `IIf(condition, truepart, falsepart)` | `IIf(Salary>100000,"High","Normal")` | Inline conditional expression.   |

These expressions can appear in any `SELECT`, `WHERE`, or `ORDER BY` clause.





## 🔗 Joins: Combining Tables

Relational databases store related data across multiple tables.
**Joins** merge those tables logically when querying.

### INNER JOIN

Returns only matching records from both tables.


In [ ]:
SELECT e.FirstName, e.LastName, d.DepartmentName
FROM Employees AS e
INNER JOIN Departments AS d
ON e.DepartmentID = d.DepartmentID;

### LEFT JOIN

- Includes all records from the left table, even if there’s no match in the right.

In [ ]:
SELECT c.CustomerName, o.OrderID
FROM Customers AS c
LEFT JOIN Orders AS o
ON c.CustomerID = o.CustomerID;


### RIGHT JOIN

Opposite of LEFT JOIN — includes all records from the right table.



### Notes on Access Join Syntax

* The Query Designer uses **visual join lines**; switching to SQL View shows equivalent JOIN statements.
* Access supports nested joins but may reformat them automatically.
* Unlike SQL Server, Access does **not** support `FULL OUTER JOIN` directly — use a UNION of LEFT and RIGHT joins.




## 📊 Grouping and Aggregation

Grouping lets you compute totals, averages, or counts across categories.


In [ ]:
SELECT Department, AVG(Salary) AS AvgSalary
FROM Employees
GROUP BY Department
HAVING AVG(Salary) > 80000;

* **GROUP BY** defines how rows are grouped.
* **Aggregate functions** (SUM, AVG, COUNT, MIN, MAX) summarize data.
* **HAVING** filters grouped results (while **WHERE** filters individual rows).

Example explanation:

> “Show departments whose average salary exceeds $80,000.”




## 🧩 Subqueries

Subqueries allow one query to feed another — useful for filters, comparisons, or calculations.

### Using IN


In [ ]:
SELECT FirstName, LastName
FROM Employees
WHERE DepartmentID IN
    (SELECT DepartmentID FROM Departments WHERE Location = "HQ");

### Using EXISTS

- Access supports nested subqueries up to several levels deep, but they can become slow on large datasets — use joins where possible.


In [ ]:
SELECT CustomerName
FROM Customers AS c
WHERE EXISTS
    (SELECT * FROM Orders AS o WHERE o.CustomerID = c.CustomerID);

## ⚡ Action Queries (Data Modification)

- Action queries change data or create new tables.

### INSERT INTO

In [ ]:
INSERT INTO Employees (FirstName, LastName, Department)
VALUES ("Jane", "Doe", "Finance");

### UPDATE

In [ ]:
UPDATE Employees
SET Salary = Salary * 1.1
WHERE Department = "Sales";

### DELETE

In [ ]:
DELETE FROM Orders
WHERE OrderDate < #1/1/2020#;


### MAKE-TABLE

- Creates a new table with results of a query.

In [ ]:
SELECT * INTO HighEarners
FROM Employees
WHERE Salary > 100000;

- Action queries are powerful — always back up before running them.


## 🧭 Parameter Queries

- Parameter queries prompt users for input dynamically.


In [ ]:
SELECT * FROM Orders
WHERE OrderDate BETWEEN [Enter Start Date:] AND [Enter End Date:];

- Access will display input boxes for `[Enter Start Date:]` and `[Enter End Date:]`.

### Executing Parameters via VBA


In [ ]:
Dim qd As DAO.QueryDef, rs As DAO.Recordset
Set qd = CurrentDb.QueryDefs("qrySalesByDate")
qd.Parameters("[Enter Start Date:]") = #1/1/2025#
qd.Parameters("[Enter End Date:]") = #1/31/2025#
Set rs = qd.OpenRecordset()

## 🧮 Domain Aggregate Functions

These functions retrieve calculated values directly from tables or queries — often used in VBA or form controls.

| Function  | Description            | Example                                   |
| --------- | ---------------------- | ----------------------------------------- |
| `DLookup` | Returns a single value | `DLookup("Salary","Employees","ID=5")`    |
| `DSum`    | Sums field values      | `DSum("Amount","Orders","CustomerID=7")`  |
| `DCount`  | Counts records         | `DCount("*","Customers","City='Boston'")` |




## 📊 Crosstab Queries (TRANSFORM)

- Crosstab queries summarize data across two dimensions, similar to Excel pivot tables.

In [ ]:
TRANSFORM Sum(Amount) AS TotalSales
SELECT Region
FROM Sales
GROUP BY Region
PIVOT Year;

- This produces a table with `Region` as rows, `Year` as columns, and total sales in the cells.

In [ ]:
SELECT Name, City FROM Customers_US
UNION ALL
SELECT Name, City FROM Customers_Canada;


## 🧱 UNION Queries

- Combine results from multiple queries with identical structures.

- Use `UNION` to remove duplicates or `UNION ALL` to include them.


## 💻 Integrating SQL with VBA

- VBA turns Access into a programmable database system.

### Executing Action Queries


In [ ]:
DoCmd.RunSQL "DELETE FROM TempData WHERE EntryDate < Date();"

### Working with Recordsets

In [ ]:
Dim rs As DAO.Recordset
Dim sql As String
sql = "SELECT * FROM Employees WHERE Department='Finance';"
Set rs = CurrentDb.OpenRecordset(sql)
Do While Not rs.EOF
    Debug.Print rs!FirstName & " " & rs!LastName
    rs.MoveNext
Loop
rs.Close

### Dynamic SQL Assembly

In [ ]:
Dim startDate As Date, endDate As Date
startDate = #1/1/2025#: endDate = #1/31/2025#
sql = "SELECT * FROM Orders WHERE OrderDate BETWEEN #" & _
       Format(startDate, "mm/dd/yyyy") & "# AND #" & Format(endDate, "mm/dd/yyyy") & "#;"
Set rs = CurrentDb.OpenRecordset(sql)

## ⚠️ Common Pitfalls and Best Practices

| Issue                     | Recommendation                                              |
| ------------------------- | ----------------------------------------------------------- |
| **Reserved Words**        | Use square brackets around names like `[Date]` or `[Name]`. |
| **Spaces in Field Names** | Always use `[Field Name]` notation.                         |
| **Wildcard Confusion**    | Use `*` and `?` — not `%` and `_`.                          |
| **Date Literals**         | Always use `#MM/DD/YYYY#`.                                  |
| **Query Performance**     | Avoid `SELECT *`; specify columns explicitly.               |
| **Data Validation**       | Use `WHERE` and parameter checks to prevent errors.         |




## 🧾 Quick Reference Tables

### Data Type Mapping

| Access Type      | SQL Equivalent | Example        |
| ---------------- | -------------- | -------------- |
| Short Text       | VARCHAR        | `"Hello"`      |
| Long Text        | MEMO           | long notes     |
| Number (Integer) | INT            | `42`           |
| Currency         | MONEY          | `12.99`        |
| Date/Time        | DATETIME       | `#2025-11-06#` |
| Yes/No           | BOOLEAN        | `True`         |



### Common Function Summary

| Category       | Function                     | Description                     |
| -------------- | ---------------------------- | ------------------------------- |
| **Text**       | `UCase(text)`                | Converts to uppercase           |
|                | `Trim(text)`                 | Removes leading/trailing spaces |
| **Date**       | `Date()`, `Now()`            | Current date/time               |
|                | `DateDiff(interval, d1, d2)` | Difference between two dates    |
| **Math**       | `Abs(x)`                     | Absolute value                  |
|                | `Rnd()`                      | Random number                   |
| **Logic**      | `IIf(cond, t, f)`            | Inline conditional              |
| **Conversion** | `CInt()`, `CDate()`          | Type conversion                 |

## 🧩 Summary

Access SQL is a **relational query language** that brings professional-grade data querying to the desktop.
When paired with VBA, it allows you to:

* Automate reports and data entry
* Build parameterized dashboards
* Perform analysis comparable to SQL Server or MySQL — on a smaller scale

- Understanding Access SQL not only deepens your Access skills but also prepares you for transitioning to enterprise databases like SQL Server.



### 📚 Further Reading

* **Microsoft Docs:** [ACE SQL Reference](https://learn.microsoft.com/en-us/office/client-developer/access/desktop-database-reference/access-sql-reference)
* **DAO Language Reference**
* **Allen Browne’s Access Tips** (excellent real-world examples)
